<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/4_multi_factor_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

4.　マルチファクター・モデルへの拡張

　実際のマーケットの現場では、多くのCAPMアノマリーと呼ばれる現象が観察されています。先行研究では、このCAPMアノマリーが生じる理由として、小型株効果、バリュー株効果、モメンタム効果などの存在を実証的に明らかにしてきました。本例では、シングルファクター・モデルを拡張し、小型株効果とバリュー株効果の変数を加えたファーマ＝フレンチの３ファクター・モデルを実装例として説明します。

　まず3ファクター・モデルに必要な小型株効果、バリュー株効果の代理変数を取得するためのパッケージをインストールします。

In [ ]:
pip install getFamaFrenchFactors

　本例では、数値分析のためのpandas、２社の株価を取得するためのpandas_datareader、datetime、回帰分析のためのstatsmodels、ファーマ＝フレンチ・ベンチマーク・データを取得するためのgetFamaFrenchFactors、の５種類のライブラリ、パッケージを読み込む。回帰分析は本章ではsklearnではなくstatsmodelsを用います。

In [41]:
#[1]ライブラリ読み込み
import pandas as pd
import pandas_datareader as data
import datetime
import statsmodels.api as sm
import getFamaFrenchFactors as gff #マルチファクターモデル・データ取得ライブラリ

　2銘柄の株価データは、テスラ（TSLA．US）と、ゼネラル・モータース（GM．US）のデータを、2012年年初から2022年末までの月次データを取得し、データフレーム名をdfとします。

In [43]:
#[2]企業株価データの取得
tickers=['TSLA.US','GM.US'] #証券コードの指定
start=datetime.date(2012,1,1)#標本期間を2012年1月1日から
end=datetime.date(2022,12,31)#2022年12月31日までに指定
df=data.DataReader(tickers,'stooq',start=start,end=end).sort_values(by='Date',ascending=True)

In [ ]:
#[3]前処理
df = df['Close'].dropna() #日次の終値のみのデータフレーム
company_list=['Tesla','GM'] #変数名再定義
df.columns = company_list
df.describe() #記述統計量の算出

　すでにインポートしたライブラリgetFamaFrenchFactors（gff）から、ファーマ＝フレンチ・ベンチマーク・データを取得します。具体的には、マーケット・ポートフォリオのリターンから無リスク資産利回りを差し引いた変数（Mkt-RF）、小型株効果の代理変数（SMB）、バリュー株効果の代理変数（HML）の３種類のデータを取得します。このデータは1926年7月から直近までのデータです。
　2銘柄の株価データと3種類のファーマ＝フレンチ・ベンチマーク・データをmergeにより統合するため、後者のデータの日時の変数名を一度リセットして、’date_ff_factors’から’Date’へ変更します。データフレーム名dfと名付けられたデータセットのうち、本分析では調整後終値の月次データを用います。この理由は、getFamaFrenchFactors（gff）から取得する3種類のファーマ＝フレンチ・ベンチマーク・データが月次データであるためです。dfのデータを前月比に換算し、欠損値を除去し、新たなデータフレーム名をReturnsとします。その後、２つのデータセットの共通変数’Date’を用いて統合し、データフレーム名をFama_French_dataと名付けます。


In [48]:
#[3]規模効果・バリュー効果データの取得
Fama_French_3=gff.famaFrench3Factor(frequency='m') #月次データの取得
Fama_French_3.rename(columns={"date_ff_factors":'Date'},inplace=True) #日付の前処理
Fama_French_3.set_index('Date',inplace=True)
Returns=df.resample('M').last().pct_change().dropna() #日次データの月次データへの変換と欠損値除去
Fama_French_data=Fama_French_3.merge(Returns,on='Date') #企業データと規模効果・バリュー効果データの統合

　整理されたデータセットを用い、以下では、最小二乗法によりテスラ（TSLA．US）と、ゼネラル・モータース（GM．US）のリスクプレミアムの3ファクター・モデルを推計します。説明変数Xはマーケット・ポートフォリオ・リター（Mkt-RF）、小型株効果の代理変数（SMB）、バリュー株効果の代理変数（HML）の３種類を採用します。被説明変数にはテスラ（TSLA．US）、ゼネラル・モーターズ（GM．US）それぞれのリターンから無リスク資産利回りを引いた値を用います。

In [46]:
#[4]マルチファクター・モデルの推計①
X=Fama_French_data[['Mkt-RF','SMB','HML']]
#X=Fama_French_data[['Mkt-RF']]
y=Fama_French_data['Tesla']-Fama_French_data['RF']
X=sm.add_constant(X)
Fama_French_Model1=sm.OLS(y,X).fit()
print(Fama_French_Model1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.199
Method:                 Least Squares   F-statistic:                     11.62
Date:                Wed, 21 Dec 2022   Prob (F-statistic):           9.11e-07
Time:                        04:35:17   Log-Likelihood:                 52.544
No. Observations:                 129   AIC:                            -97.09
Df Residuals:                     125   BIC:                            -85.65
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0328      0.015      2.208      0.0

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [47]:
#[5]マルチファクター・モデルの推計②
X=Fama_French_data[['Mkt-RF','SMB','HML']]
z=Fama_French_data['GM']-Fama_French_data['RF']
X=sm.add_constant(X)
Fama_French_Model2=sm.OLS(z,X).fit()
print(Fama_French_Model2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     46.12
Date:                Wed, 21 Dec 2022   Prob (F-statistic):           3.89e-20
Time:                        04:35:24   Log-Likelihood:                 178.05
No. Observations:                 129   AIC:                            -348.1
Df Residuals:                     125   BIC:                            -336.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0026      0.006     -0.465      0.6

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
